In [6]:
!pip install streamlit pandas openai supabase pyngrok python-dotenv bcrypt cloudflare
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


(Reading database ... 121717 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.11.1) over (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...
(Reading database ... 121717 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.11.1) over (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [7]:
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY"
os.environ["SUPABASE_URL"] = "YOUR_SUPABASE_URL"
os.environ["SUPABASE_KEY"] = "YOUR_SUPABASE_KEY"


🔐 Environment variables set.


In [8]:
print("OpenAI Key:", os.environ.get("OPENAI_API_KEY") is not None)
print("Supabase URL:", os.environ.get("SUPABASE_URL"))
print("Supabase Key:", os.environ.get("SUPABASE_KEY") is not None)


OpenAI Key: True
Supabase URL: https://svfejtwsciszcjwcnshh.supabase.co
Supabase Key: True


In [9]:
%%writefile utils.py
import os
import bcrypt
import pandas as pd
import openai
from supabase import create_client, Client

# Load environment keys
openai.api_key = os.getenv("OPENAI_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# Connect to Supabase (Optional Mode)
db_enabled = SUPABASE_URL not in [None, "", "YOUR_SUPABASE_URL"]
supabase = create_client(SUPABASE_URL, SUPABASE_KEY) if db_enabled else None


# ---------------- AUTH ----------------
def register_user(email, password):
    if not db_enabled:
        return None, "⚠ Database disabled — running in demo mode."

    hashed_pw = bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()
    exists = supabase.table("users").select("*").eq("email", email).execute()

    if exists.data:
        return None, "User already exists."

    res = supabase.table("users").insert({"email": email, "password_hash": hashed_pw}).execute()
    return res.data, None


def login_user(email, password):
    if not db_enabled:
        return {"email": email, "id": "demo_user"}, None

    res = supabase.table("users").select("*").eq("email", email).execute()

    if not res.data or len(res.data) == 0:
        return None, "User not found."

    user_data = res.data[0]  # safely get the first row

    if bcrypt.checkpw(password.encode(), user_data["password_hash"].encode()):
        return user_data, None
    return None, "Incorrect password."


# ---------------- AI NORMALIZATION ----------------
def normalize_text(text):
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Rewrite insurance claim text professionally, standardized and clean."},
                {"role": "user", "content": text}
            ]
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"[AI ERROR] {e}"


# ---------------- SAVE HISTORY ----------------
def save_to_history(user_id, original, normalized):
    if db_enabled:
        supabase.table("claims").insert({
            "user_id": user_id,
            "original_text": original,
            "normalized_text": normalized
        }).execute()


def fetch_history(user_id):
    if not db_enabled:
        return []
    return supabase.table("claims").select("*").eq("user_id", user_id).execute().data


Overwriting utils.py


In [10]:
%%writefile app.py
import streamlit as st
import pandas as pd
from utils import login_user, register_user, normalize_text, save_to_history, fetch_history

st.set_page_config(page_title="Claims Normalizer", layout="wide")

st.title("🧾 Claims Description Normalizer Using AI")

if "user" not in st.session_state:
    st.session_state.user = None


# ---------------- LOGIN UI ----------------
if st.session_state.user is None:
    st.subheader("🔐 Login or Sign Up")

    mode = st.radio("Select mode", ["Login", "Sign Up"])
    email = st.text_input("Email")
    pwd = st.text_input("Password", type="password")

    if st.button("Continue"):
        if mode == "Sign Up":
            _, err = register_user(email, pwd)
            if err:
                st.error(err)
            else:
                st.success("Account created. Now login.")
        else:
            user, err = login_user(email, pwd)
            if err:
                st.error(err)
            else:
                st.session_state.user = user
                st.rerun()
    st.stop()


# ---------------- MAIN APP ----------------
st.success(f"Welcome, {st.session_state.user['email']} 👍")

uploaded = st.file_uploader("📁 Upload Claims CSV", type=["csv"])

if uploaded:
    df = pd.read_csv(uploaded)
    st.write("Data Loaded Successfully:")
    st.dataframe(df)

    column = st.selectbox("Select Claim Description Column", df.columns)

    if st.button("🚀 Normalize Text"):
        st.info("Processing... please wait")
        progress = st.progress(0)

        results = []
        for idx, text in df[column].items():
            cleaned = normalize_text(text)
            results.append(cleaned)
            save_to_history(st.session_state.user["id"], text, cleaned)
            progress.progress((idx+1)/len(df))

        df["Normalized_Claim"] = results
        st.success("✔ Completed!")
        st.dataframe(df)

        st.download_button("⬇ Download Result CSV", df.to_csv(index=False), "normalized.csv")


# ---------------- HISTORY ----------------
st.subheader("📊 Your Processing History")

history = fetch_history(st.session_state.user["id"])

if history:
    st.dataframe(pd.DataFrame(history))
else:
    st.info("No history found.")


Overwriting app.py


In [ ]:
!streamlit run app.py --server.port 8501 &>/content/log.txt &
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


2025-12-03T15:13:29Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-03T15:13:29Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-03T15:13:34Z INF +--------------------------------------------------------------------------------------------+
2025-12-03T15:13:34Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-12-03T15:13:34Z INF |  https://usa-excited-brief-dispatched.trycloudflare.co